<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-27 15:04:09
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.34 C
-------------------
Today PnL: 30.50 K (0.23%)
Current PnL: -15.23 L (-10.61%)
CY Booked + Current PnL: -8.49 L (-5.92%)
-------------------
Total profit:  3.58 L
Total loss:  -18.81 L
-------------------
Total Booked + Current PnL: 18.49 L (15.86%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (5.02%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.07 C
Est FTT PnL: 72.55 L (54.01%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 9.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,52.69,102418.0,24056.0,9976.0,0.92,30.70,9.74,43.43,77.0,2.41,0.76,37.65,XY24,NTT,MISC
36,HINDALCO,651.95,761.55,-19.08,H-LC,74.19,111488.0,7176.0,10357.0,0.90,6.88,9.29,16.81,27.0,0.69,0.83,24.03,X5K,ATH,METALS
84,UJJIVANSFB,52.77,53.00,51.69,M-SC,83.87,131409.0,-11070.0,11695.0,-0.88,-7.77,8.90,0.44,243.0,-0.95,0.98,55.10,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.00,-15.29,X-MC,37.63,88928.0,-13724.0,13722.0,1.24,-13.37,15.43,-0.00,58.0,-1.00,0.66,35.18,OX40N,NTT,IT
89,VIPIND,488.80,489.00,-212.78,H-SC,96.77,81315.0,-14001.0,14043.0,0.07,-14.69,17.27,0.04,154.0,-1.00,0.61,60.38,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,89.46,M-MC,51.61,227149.0,2187.0,165478.0,-1.28,0.97,72.85,74.53,185.0,0.01,1.69,31.88,XY24,BTT,STEEL
12,BANKINDIA,116.91,190.00,-31.29,M-MC,57.53,179746.0,-62.0,112467.0,-0.07,-0.03,62.57,62.52,171.0,-0.00,1.34,34.04,XR,NTT,BANKS
51,KPIGREEN,497.21,732.16,21.90,H-SC,70.97,124753.0,-544.0,59757.0,-0.42,-0.43,47.90,47.25,139.0,-0.01,0.93,56.07,MH,ATH,POWER
35,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,32.80,151406.0,493.0,59957.0,1.12,0.33,39.60,40.06,102.0,0.01,1.13,23.84,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,JIOFIN,310.58,387.00,5.62,M-LC,89.25,226275.0,8869.0,44621.0,3.44,4.08,19.72,24.61,95.0,0.20,1.69,60.88,XY24,BTT,FINANCE
20,CERA,8421.60,8422.00,-14.32,X-SC,69.89,88439.0,-21042.0,21048.0,3.13,-19.22,23.80,0.00,68.0,-1.00,0.66,33.85,OX40N,NTT,CERAMICS
57,NATIONALUM,189.63,247.44,-48.64,H-MC,65.59,102603.0,2099.0,28544.0,3.01,2.09,27.82,30.49,97.0,0.07,0.76,35.17,MH,ATH,METALS
5,APOLLOHOSP,6904.43,8285.00,-19.51,H-LC,53.76,109650.0,6084.0,14627.0,2.94,5.87,13.34,20.00,30.0,0.42,0.82,20.77,X40N,BTT,HEALTHCARE
6,ASIANPAINT,2961.56,4250.00,-19.25,H-LC,49.46,160269.0,-41117.0,128728.0,2.93,-20.42,80.32,43.51,32.0,-0.32,1.19,11.56,X40,BTT,PAINTS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,90.32,69658.0,-43891.0,83945.0,-3.69,-38.65,120.51,35.27,261.0,-0.52,0.52,70.49,XR,NTT,HOTELS
61,RAJESHEXPO,517.65,518.00,1828.03,M-SC,38.71,52621.0,-84556.0,84651.0,-2.33,-61.64,160.87,0.07,231.0,-1.00,0.39,30.87,OX40N,NTT,JEWELLERY
17,BSOFT,628.85,844.81,-9.72,M-SC,59.14,75714.0,-34335.0,72125.0,-2.16,-31.20,95.26,34.34,200.0,-0.48,0.56,24.65,XR,ATH,IT
15,BLUESTARCO,1646.70,2337.55,-13.09,H-SC,64.52,162910.0,-1760.0,70850.0,-1.91,-1.07,43.49,41.95,121.0,-0.02,1.21,6.32,X40N,ATH,AC
47,JCHAC,2282.24,2282.00,19921.74,M-SC,48.39,97888.0,-29917.0,29905.0,-1.91,-23.41,30.55,-0.01,222.0,-1.00,0.73,8.64,OX40N,NTT,AC


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,51.69,M-SC,83.87,131409.0,-11070.0,11695.0,-0.88,-7.77,8.90,0.44,243.0,-0.95,0.98,55.10,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.0,-15.29,X-MC,37.63,88928.0,-13724.0,13722.0,1.24,-13.37,15.43,-0.00,58.0,-1.00,0.66,35.18,OX40N,NTT,IT
89,VIPIND,488.80,489.0,-212.78,H-SC,96.77,81315.0,-14001.0,14043.0,0.07,-14.69,17.27,0.04,154.0,-1.00,0.61,60.38,OX40N,NTT,MISC
76,SYMPHONY,1306.42,1306.0,-33.55,M-SC,1.08,141742.0,-29399.0,29341.0,0.15,-17.18,20.70,-0.03,189.0,-1.00,1.06,1.87,OX40N,NTT,DURABLES
50,KANSAINER,299.63,340.0,-68.02,H-SC,22.58,223290.0,-46377.0,82707.0,1.64,-17.20,37.04,13.47,142.0,-0.56,1.66,12.35,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-17.74,X-LC,30.11,268414.0,-27520.0,121430.0,-0.02,-9.30,45.24,31.73,1.0,-0.23,2.00,7.44,X40,BTT,IT
45,INFY,1461.46,2275.00,-10.27,X-LC,57.53,287778.0,26177.0,119457.0,-0.51,10.01,41.51,55.67,2.0,0.22,2.14,16.72,X40,BTT,IT
37,HINDUNILVR,2413.81,2723.00,-17.52,X-LC,24.73,258317.0,-12030.0,46652.0,1.12,-4.45,18.06,12.81,6.0,-0.26,1.92,8.03,XY25,NTT,FMCG
58,NESTLEIND,2268.60,2755.00,-9.43,X-LC,35.48,287247.0,21821.0,35102.0,0.97,8.22,12.22,21.44,9.0,0.62,2.14,14.93,XY25,NTT,FMCG
16,BRITANNIA,4983.33,6446.05,16.13,X-LC,67.74,266409.0,37176.0,30104.0,-0.63,16.22,11.30,29.35,10.0,1.23,1.98,26.58,XY25,ATH,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.0,-37.55,M-SC,91.40,205976.0,34355.0,127705.0,-0.45,20.02,62.00,94.43,198.0,0.27,1.53,52.02,XR,NTT,DURABLES
85,UNIONBANK,123.87,163.0,-2.93,M-LC,73.12,169299.0,28459.0,16033.0,0.53,20.21,9.47,31.59,89.0,1.78,1.26,47.06,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.03,M-LC,52.69,102418.0,24056.0,9976.0,0.92,30.70,9.74,43.43,77.0,2.41,0.76,37.65,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,89.46,M-MC,51.61,227149.0,2187.0,165478.0,-1.28,0.97,72.85,74.53,185.0,0.01,1.69,31.88,XY24,BTT,STEEL
19,CAMS,3643.00,5250.99,6.98,H-SC,82.80,120148.0,18144.0,26877.0,1.86,17.79,22.37,44.14,124.0,0.68,0.89,36.92,X40N,ATH,MISC
48,JIOFIN,310.58,387.00,5.62,M-LC,89.25,226275.0,8869.0,44621.0,3.44,4.08,19.72,24.61,95.0,0.20,1.69,60.88,XY24,BTT,FINANCE
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,52.69,102418.0,24056.0,9976.0,0.92,30.70,9.74,43.43,77.0,2.41,0.76,37.65,XY24,NTT,MISC
85,UNIONBANK,123.87,163.00,-2.93,M-LC,73.12,169299.0,28459.0,16033.0,0.53,20.21,9.47,31.59,89.0,1.78,1.26,47.06,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-37.55,M-SC,91.40,205976.0,34355.0,127705.0,-0.45,20.02,62.00,94.43,198.0,0.27,1.53,52.02,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-4.22,M-LC,78.49,164319.0,13374.0,96077.0,-1.30,8.86,58.47,72.51,69.0,0.14,1.22,13.19,XR,NTT,IT
42,INDIAMART,2327.09,4850.92,-46.77,H-SC,93.55,138436.0,15100.0,118667.0,-0.96,12.24,85.72,108.45,122.0,0.13,1.03,37.33,AR,ATH,MISC
57,NATIONALUM,189.63,247.44,-48.64,H-MC,65.59,102603.0,2099.0,28544.0,3.01,2.09,27.82,30.49,97.0,0.07,0.76,35.17,MH,ATH,METALS
35,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,32.80,151406.0,493.0,59957.0,1.12,0.33,39.60,40.06,102.0,0.01,1.13,23.84,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,90.32,69658.0,-43891.0,83945.0,-3.69,-38.65,120.51,35.27,261.0,-0.52,0.52,70.49,XR,NTT,HOTELS
55,MASFIN,326.60,402.39,-17.80,H-SC,77.42,93210.0,-4770.0,27506.0,0.57,-4.87,29.51,23.21,137.0,-0.17,0.69,35.19,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,4.24,X-SC,98.92,97064.0,-3899.0,98433.0,-0.57,-3.86,101.41,93.63,64.0,-0.04,0.72,44.57,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,52.69,102418.0,24056.0,9976.0,0.92,30.70,9.74,43.43,77.0,2.41,0.76,37.65,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-19.51,H-LC,53.76,109650.0,6084.0,14627.0,2.94,5.87,13.34,20.00,30.0,0.42,0.82,20.77,X40N,BTT,HEALTHCARE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7277.78,M-SC,100.00,80922.0,-12888.0,89508.0,0.37,-13.74,110.61,81.67,242.0,-0.14,0.60,56.71,XR,NTT,CERAMICS
4,ALKYLAMINE,2347.98,4546.37,4.24,X-SC,98.92,97064.0,-3899.0,98433.0,-0.57,-3.86,101.41,93.63,64.0,-0.04,0.72,44.57,SR,ATH,CHEMICALS
49,JPPOWER,18.73,26.20,-31.87,L-SC,97.85,137223.0,-4619.0,61188.0,0.83,-3.26,44.59,39.88,259.0,-0.08,1.02,39.06,XY24,NTT,POWER
89,VIPIND,488.80,489.00,-212.78,H-SC,96.77,81315.0,-14001.0,14043.0,0.07,-14.69,17.27,0.04,154.0,-1.00,0.61,60.38,OX40N,NTT,MISC
66,SAMMAANCAP,170.35,326.00,-172.73,M-SC,95.70,84378.0,-17832.0,111219.0,-1.91,-17.45,131.81,91.37,206.0,-0.16,0.63,36.97,XY25,NTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.23
1,25,42.45
2,50,71.04


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.82
LC    34.46
MC    21.64
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.28
X40      12.37
XY25     12.15
X40N     11.88
XR       10.30
OX40N     8.25
AR        7.87
MH        1.69
X5K       1.50
X200      1.42
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.77
M-SC    16.19
H-LC    12.67
X-LC    12.35
H-MC     8.65
M-LC     8.36
X-MC     7.64
M-MC     4.99
X-SC     4.39
L-SC     2.47
L-LC     1.08
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.33,-1.59,35.59
IT,11.91,-10.97,66.09
FINANCE,7.97,-12.69,53.47
BANKS,7.95,-6.82,50.41
MISC,6.65,-20.97,68.15
PAINTS,5.68,-14.36,36.90
ELECTRICAL,5.06,-3.42,48.21
HEALTHCARE,4.52,-5.99,36.95
AUTO,4.41,-14.18,64.38


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2729695.0,25
XR,1074738.0,13
AR,1012103.0,9
X40,657512.0,9
X40N,568779.0,11
XY25,485423.0,8
OX40N,322303.0,11
SR,180921.0,2
MH,88301.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1993192.0,20
M-SC,1765755.0,21
H-LC,567445.0,11
X-LC,516123.0,7
M-MC,498069.0,4
H-MC,474496.0,7
X-MC,425963.0,6
X-SC,329002.0,5
M-LC,321384.0,6


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,868630.0,6
M-SC,XY24,816106.0,7
H-SC,AR,518612.0,3
M-MC,XY24,385602.0,3
M-SC,XR,330092.0,4
X-LC,X40,240887.0,2
H-SC,X40N,236652.0,4
H-MC,XY24,225921.0,2
H-SC,XR,214133.0,3


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
